# Step 1

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from vivarium import InteractiveContext, Artifact
from pathlib import Path
#import itertools
import pandas as pd, numpy as np

In [2]:
pip list | grep vivarium

vivarium                  2.1.1
vivarium-nih-us-cvd       0.1.dev372+gf9e9c54
vivarium-public-health    2.1.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip freeze | grep vivarium

vivarium==2.1.1
vivarium-nih-us-cvd @ git+https://github.com/ihmeuw/vivarium_nih_us_cvd.git@f9e9c54a34f395aaa9b4dfa24c962f413cb50005
vivarium-public-health==2.1.0
Note: you may need to restart the kernel to use updated packages.


# Step 2

In [4]:
import vivarium_nih_us_cvd
# model_specifications/model_spec.yaml is the typical location within the engineering repo for the
# default spec to live. Ask the engineers if you can't find it.
path = Path(vivarium_nih_us_cvd.__file__.replace('__init__.py', 'model_specifications/nih_us_cvd.yaml'))

from vivarium.framework.configuration import build_model_specification

custom_model_specification = build_model_specification(path)

# Customizations: will be specific to your model and use case!
# Delete observer configuration
del custom_model_specification.configuration.observers
# Change population size
custom_model_specification.configuration.population.population_size = 50_000

# Delete the "metrics" component (an observer)
del custom_model_specification.components.vivarium_public_health.metrics

## Filter out components with 'Observer' or 'Stratifier' in the name
custom_model_specification.components.vivarium_nih_us_cvd.components = [
    c for c in custom_model_specification.components.vivarium_nih_us_cvd.components
    if 'Observer' not in c and 'Stratifier' not in c
]

custom_model_specification.configuration.input_data.artifact_path = '/mnt/team/simulation_science/pub/models/vivarium_nih_us_cvd/artifacts/vv-variable-coverage/alabama.hdf'

# Step 3

In [5]:
sim = InteractiveContext(custom_model_specification)

2023-12-15 13:55:24.780 | INFO     | simulation_1-artifact_manager:67 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_nih_us_cvd/artifacts/vv-variable-coverage/alabama.hdf.
2023-12-15 13:55:24.782 | INFO     | simulation_1-artifact_manager:68 - Artifact base filter terms are ['draw == 0'].
2023-12-15 13:55:24.784 | INFO     | simulation_1-artifact_manager:69 - Artifact additional filter terms are None.
2023-12-15 13:56:46.683 | WARNING  | simulation_1-values_manager:279 - Unsourced pipelines: ['disability_weight']
2023-12-15 13:56:46.691 | WARNING  | simulation_1-resource_manager:256 - Resource missing_value_source.disability_weight is not provided by any component but is needed to compute (value.disability_weight).


# Step 4

In [6]:
pop0 = sim.get_population()
pop0.head()

,tracked,entrance_time,sex,exit_time,location,alive,age,cause_of_death,years_of_life_lost,high_ldl_cholesterol_propensity,...,discontinued_sbp_medication,sbp_therapeutic_inertia_constant_component,lifestyle,outreach,ldlc_medication_adherence,lifestyle_adherence,sbp_medication_start_date,last_fpg_test_date,scheduled_date,visit_type
0,True,2020-12-04,Female,NaT,Alabama,alive,47.388882,not_dead,0.0,0.173110,...,False,-1.163186,NaT,cat2,cat3,False,2018-11-01 10:03:25.507035528,NaT,NaT,none
1,True,2020-12-04,Male,NaT,Alabama,alive,41.564194,not_dead,0.0,0.187950,...,False,0.597742,NaT,cat2,cat3,True,2019-10-27 23:08:06.352339272,NaT,NaT,none
2,True,2020-12-04,Female,NaT,Alabama,alive,53.155826,not_dead,0.0,0.168561,...,False,0.352270,NaT,cat2,cat3,False,2019-11-17 02:07:58.816891772,NaT,NaT,none
3,True,2020-12-04,Male,NaT,Alabama,alive,30.457692,not_dead,0.0,0.192373,...,False,0.274154,NaT,cat2,cat1,True,2018-09-20 09:01:55.751936592,NaT,NaT,none
4,True,2020-12-04,Female,NaT,Alabama,alive,19.554806,not_dead,0.0,0.179608,...,False,0.350752,NaT,cat2,cat3,False,NaT,NaT,NaT,none


In [7]:
pop0.columns

Index(['tracked', 'entrance_time', 'sex', 'exit_time', 'location', 'alive',
       'age', 'cause_of_death', 'years_of_life_lost',
       'high_ldl_cholesterol_propensity',
       'high_fasting_plasma_glucose_propensity',
       'high_body_mass_index_in_adults_propensity',
       'high_systolic_blood_pressure_propensity', 'ischemic_stroke',
       'ischemic_heart_disease_and_heart_failure',
       'ensemble_propensity_risk_factor.high_ldl_cholesterol',
       'ensemble_propensity_risk_factor.high_systolic_blood_pressure',
       'ensemble_propensity_risk_factor.high_body_mass_index_in_adults',
       'ensemble_propensity_risk_factor.high_fasting_plasma_glucose',
       'sbp_medication_adherence_propensity',
       'ldlc_medication_adherence_propensity', 'outreach_propensity',
       'polypill_propensity', 'lifestyle_propensity',
       'susceptible_to_ischemic_stroke_event_time',
       'susceptible_to_ischemic_stroke_event_count',
       'acute_ischemic_stroke_event_time', 'acute_ische

In [8]:
sim.list_values()

['metrics',
 'cause_specific_mortality_rate',
 'acute_ischemic_stroke.incidence_rate',
 'acute_ischemic_stroke.incidence_rate.paf',
 'acute_ischemic_stroke.dwell_time',
 'acute_ischemic_stroke.disability_weight',
 'disability_weight',
 'acute_ischemic_stroke.excess_mortality_rate',
 'acute_ischemic_stroke.excess_mortality_rate.paf',
 'mortality_rate',
 'chronic_ischemic_stroke.dwell_time',
 'chronic_ischemic_stroke.disability_weight',
 'chronic_ischemic_stroke.excess_mortality_rate',
 'chronic_ischemic_stroke.excess_mortality_rate.paf',
 'chronic_ischemic_stroke_to_acute_ischemic_stroke.transition_rate',
 'chronic_ischemic_stroke_to_acute_ischemic_stroke.transition_rate.paf',
 'acute_myocardial_infarction.incidence_rate',
 'heart_failure_from_ischemic_heart_disease.incidence_rate',
 'heart_failure_residual.incidence_rate',
 'acute_myocardial_infarction.incidence_rate.paf',
 'heart_failure_from_ischemic_heart_disease.incidence_rate.paf',
 'heart_failure_residual.incidence_rate.paf',
 's

In [9]:
data1 = pd.concat([pop0,
                   sim.get_value('high_ldl_cholesterol.exposure')(pop0.index).rename('high_ldl'),
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp'),
                  ], axis=1)
data1.head()

,tracked,entrance_time,sex,exit_time,location,alive,age,cause_of_death,years_of_life_lost,high_ldl_cholesterol_propensity,...,lifestyle,outreach,ldlc_medication_adherence,lifestyle_adherence,sbp_medication_start_date,last_fpg_test_date,scheduled_date,visit_type,high_ldl,high_sbp
0,True,2020-12-04,Female,NaT,Alabama,alive,47.388882,not_dead,0.0,0.173110,...,NaT,cat2,cat3,False,2018-11-01 10:03:25.507035528,NaT,NaT,none,1.872148,139.628132
1,True,2020-12-04,Male,NaT,Alabama,alive,41.564194,not_dead,0.0,0.187950,...,NaT,cat2,cat3,True,2019-10-27 23:08:06.352339272,NaT,NaT,none,1.756671,138.432993
2,True,2020-12-04,Female,NaT,Alabama,alive,53.155826,not_dead,0.0,0.168561,...,NaT,cat2,cat3,False,2019-11-17 02:07:58.816891772,NaT,NaT,none,1.854479,142.957697
3,True,2020-12-04,Male,NaT,Alabama,alive,30.457692,not_dead,0.0,0.192373,...,NaT,cat2,cat1,True,2018-09-20 09:01:55.751936592,NaT,NaT,none,2.090760,131.928065
4,True,2020-12-04,Female,NaT,Alabama,alive,19.554806,not_dead,0.0,0.179608,...,NaT,cat2,cat3,False,NaT,NaT,NaT,none,0.000000,50.000000


# Step 5

In [10]:
sim_data = pd.DataFrame()
for step in list(range(0,5)):
    sim.step()
    pop_t = sim.get_population()
    data_t = pd.concat([pop_t, 
                   sim.get_value('high_ldl_cholesterol.exposure')(pop0.index).rename('high_ldl'),
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp')],
                       axis=1)
    data_t['step'] = step
    sim_data = pd.concat([sim_data, data_t])

KeyboardInterrupt: 

In [ ]:
sim_data